## Motivation

I got tired and bored in doing this: https://www.kaggle.com/cashfeg/play-game-by-hand-2-days  
So I decided to think of easier way to do.  

Reference: 
- Turorial: https://www.kaggle.com/stonet2000/lux-ai-season-1-jupyter-notebook-tutorial
- API: https://github.com/Lux-AI-Challenge/Lux-Design-2021/blob/master/kits/README.md
- lux-ai-specifications: https://www.kaggle.com/c/lux-ai-2021/overview/lux-ai-specifications

## Imports
see [tutorial notebook](https://www.kaggle.com/stonet2000/lux-ai-season-1-jupyter-notebook-tutorial) for details. 

In [ ]:
!pip install kaggle-environments -U

In [ ]:
# run this if using kaggle notebooks
!cp -r ../input/lux-ai-2021/* .

Use same game seed so that I can compare. 

In [ ]:
from kaggle_environments import make
# pick interesting seed shown in tutorial
# https://www.kaggle.com/stonet2000/lux-ai-season-1-jupyter-notebook-tutorial
env = make("lux_ai_2021", configuration={"seed": 562124210, "loglevel": 2}, debug=True)
# env = make("lux_ai_2021", configuration={"seed": 562124210, "loglevel": 0}, debug=True)

In [ ]:
from lux.game import Game
from lux.game_map import Cell, RESOURCE_TYPES, Position
from lux.constants import Constants
from lux.game_constants import GAME_CONSTANTS
from lux import annotate
import networkx as nx
import math
import sys
import random

# My Commands for smart agents
What makes me tired and bored is micro-management. It is tiring, even if they listen to what I say completely.  
So I decided to make the agent a litte smarter.   
Instead of "move north, stay, move west, build city, ...", I want to say "make your city bigger" or "go there and build a new city"

## My basic commands fot units 

## commands for workers and carts

In [ ]:
north = ("move", Constants.DIRECTIONS.NORTH)
east = ("move", Constants.DIRECTIONS.EAST)
south = ("move", Constants.DIRECTIONS.SOUTH)
west = ("move", Constants.DIRECTIONS.WEST)
center = ("move", Constants.DIRECTIONS.CENTER)
city = ("city", )
# transfer = ("transfer", dest_id, resourceType, amount)
pillage = ("pillage", )

## commands for citytiles

In [ ]:
research = ("research", )
worker = ("worker", )
cart = ("cart", )

## commands for smart workers and carts

In [ ]:
# traveller = ("traveller", pos)
# citizen = ("citizen", city)

In [ ]:
smart_worker = ("smart_worker", )
smart_cart = ("smart_cart", )

In [ ]:
# snippet to find the closest city tile to a position
def find_closest_city_tile(pos, player):
    closest_city_tile = None
    if len(player.cities) > 0:
        closest_dist = math.inf
        # the cities are stored as a dictionary mapping city id to the city object, which has a citytiles field that
        # contains the information of all citytiles in that city
        for k, city in player.cities.items():
            for city_tile in city.citytiles:
                dist = city_tile.pos.distance_to(pos)
                if dist < closest_dist:
                    closest_dist = dist
                    closest_city_tile = city_tile
    return closest_city_tile

In [ ]:
# this snippet finds all resources stored on the map and puts them into a list so we can search over them
def find_resources(game_state):
    resource_tiles: list[Cell] = []
    width, height = game_state.map_width, game_state.map_height
    for y in range(height):
        for x in range(width):
            cell = game_state.map.get_cell(x, y)
            if cell.has_resource():
                resource_tiles.append(cell)
    return resource_tiles

# the next snippet finds the closest resources that we can mine given position on a map
def find_closest_resources(pos, player, resource_tiles):
    closest_dist = math.inf
    closest_resource_tile = None
    for resource_tile in resource_tiles:
        # we skip over resources that we can't mine due to not having researched them
        if resource_tile.resource.type == Constants.RESOURCE_TYPES.COAL and not player.researched_coal(): continue
        if resource_tile.resource.type == Constants.RESOURCE_TYPES.URANIUM and not player.researched_uranium(): continue
        dist = resource_tile.pos.distance_to(pos)
        if dist < closest_dist:
            closest_dist = dist
            closest_resource_tile = resource_tile
    return closest_resource_tile

## function to return command

In [ ]:
class City(object):
    
    def __init__(self, observation, pos_list):
        self.game_state = Game()
        self.game_state._initialize(observation["updates"])
        self.game_state._update(observation["updates"][2:])
        self.game_state.id = observation.player
        self.width, self.height = self.game_state.map.width, self.game_state.map.height
        
        self.player = self.game_state.players[observation.player]
        self.opponent = self.game_state.players[(observation.player + 1) % 2]
        self.resource_tiles = find_resources(self.game_state)
        
        self.pos_list = pos_list
        self.city_tiles = []
        self.city_tiles_to_build = []
        for pos in pos_list:
            tile = self.game_state.map.get_cell_by_pos(pos)
            if tile.citytile is None:  # not already a citytile
                if not tile.has_resource():  # not resource
                    self.city_tiles_to_build.append(tile)
            else:
                self.city_tiles.append(tile)
        
    def update(self, observation):
        self.game_state._update(observation["updates"])
        
        self.player = self.game_state.players[observation.player]
        self.opponent = self.game_state.players[(observation.player + 1) % 2]
        self.resource_tiles = find_resources(self.game_state)
        
        self.city_tiles = []
        self.city_tiles_to_build = []
        for pos in self.pos_list:
            tile = self.game_state.map.get_cell_by_pos(pos)
            if tile.citytile is None:  # not already a citytile
                if not tile.has_resource():  # not resource
                    self.city_tiles_to_build.append(tile)
            else:
                self.city_tiles.append(tile)
                
    def update_pos_list(self, pos_list):
        self.pos_list = pos_list
        self.city_tiles = []
        self.city_tiles_to_build = []
        for pos in pos_list:
            tile = self.game_state.map.get_cell_by_pos(pos)
            if tile.citytile is None:  # not already a citytile
                if not tile.has_resource():  # not resource
                    self.city_tiles_to_build.append(tile)
            else:
                self.city_tiles.append(tile)
                
    def find_closest_resources(self, pos):
        closest_dist = math.inf
        closest_resource_tile = None
        for resource_tile in self.resource_tiles:
            # we skip over resources that we can't mine due to not having researched them
            if resource_tile.resource.type == Constants.RESOURCE_TYPES.COAL and not self.player.researched_coal(): continue
            if resource_tile.resource.type == Constants.RESOURCE_TYPES.URANIUM and not self.player.researched_uranium(): continue
            dist = resource_tile.pos.distance_to(pos)
            if dist < closest_dist:
                closest_dist = dist
                closest_resource_tile = resource_tile
        return closest_resource_tile

    def distance_to_resources(self, pos):
        closest_dist = 1000
        closest_resource_tile = None
        for resource_tile in self.resource_tiles:
            # we skip over resources that we can't mine due to not having researched them
            if resource_tile.resource.type == Constants.RESOURCE_TYPES.COAL and not self.player.researched_coal(): continue
            if resource_tile.resource.type == Constants.RESOURCE_TYPES.URANIUM and not self.player.researched_uranium(): continue
            dist = resource_tile.pos.distance_to(pos)
            if dist < closest_dist:
                closest_dist = dist
                closest_resource_tile = resource_tile
        return closest_dist
    
    def find_closest_city_tile(self, pos):
        closest_city_tile = None
        if len(self.city_tiles) > 0:
            closest_dist = math.inf
            for city_tile in self.city_tiles:
                dist = city_tile.pos.distance_to(pos)
                if dist < closest_dist:
                    closest_dist = dist
                    closest_city_tile = city_tile
        return closest_city_tile
    
    def distance_to_city_tile(self, pos):
        closest_city_tile = None
        closest_dist = 1000
        if len(self.city_tiles) > 0:
            closest_dist = 1000
            for city_tile in self.city_tiles:
                dist = city_tile.pos.distance_to(pos)
                if dist < closest_dist:
                    closest_dist = dist
                    closest_city_tile = city_tile
        return closest_dist
    
    def find_empty_city_tile(self, pos):
        closest_city_tile = None
        if len(self.city_tiles_to_build) > 0:
            closest_dist = math.inf
            for city_tile in self.city_tiles_to_build:
                dist = city_tile.pos.distance_to(pos)
                if dist < closest_dist:
                    closest_dist = dist
                    closest_city_tile = city_tile
        return closest_city_tile
    
    def distance_to_empty_city_tile(self, pos):
        closest_city_tile = None
        closest_dist = 1000
        if len(self.city_tiles_to_build) > 0:
            closest_dist = 1000
            for city_tile in self.city_tiles_to_build:
                dist = city_tile.pos.distance_to(pos)
                if dist < closest_dist:
                    closest_dist = dist
                    closest_city_tile = city_tile
        return closest_dist
    
    def worker_action(self, unit):
        # returns list of (pos_x, pos_y, score, action)
        res_list = []
        pos_0 = unit.pos
        pos_1 = unit.pos.translate(Constants.DIRECTIONS.NORTH, 1)
        pos_2 = unit.pos.translate(Constants.DIRECTIONS.EAST, 1)
        pos_3 = unit.pos.translate(Constants.DIRECTIONS.SOUTH, 1)
        pos_4 = unit.pos.translate(Constants.DIRECTIONS.WEST, 1)
        
        d0_empty_city = self.distance_to_empty_city_tile(pos_0)
        d1_empty_city = self.distance_to_empty_city_tile(pos_1)
        d2_empty_city = self.distance_to_empty_city_tile(pos_2)
        d3_empty_city = self.distance_to_empty_city_tile(pos_3)
        d4_empty_city = self.distance_to_empty_city_tile(pos_4)
        
        d0_city = self.distance_to_city_tile(pos_0)
        d1_city = self.distance_to_city_tile(pos_1)
        d2_city = self.distance_to_city_tile(pos_2)
        d3_city = self.distance_to_city_tile(pos_3)
        d4_city = self.distance_to_city_tile(pos_4)
        
        d0_resource = self.distance_to_resources(pos_0)
        d1_resource = self.distance_to_resources(pos_1)
        d2_resource = self.distance_to_resources(pos_2)
        d3_resource = self.distance_to_resources(pos_3)
        d4_resource = self.distance_to_resources(pos_4)
        
        v0 = 0
        v1 = 0
        v2 = 0
        v3 = 0
        v4 = 0
        
        # build city
        if unit.get_cargo_space_left() == 0:
            if d0_empty_city == 1000:
                pass
            else:
                if d0_empty_city == 0:
                    res_list.append((pos_0.x, pos_0.y, 1000, unit.build_city()))
                else:
                    v0 += 900 - 100 * d0_empty_city
                    v1 += 900 - 100 * d1_empty_city
                    v2 += 900 - 100 * d2_empty_city
                    v3 += 900 - 100 * d3_empty_city
                    v4 += 900 - 100 * d4_empty_city
            # may hit to a city, but that's better than wandering
        else:
            v0 += - 5 * d0_empty_city
            v1 += - 5 * d1_empty_city
            v2 += - 5 * d2_empty_city
            v3 += - 5 * d3_empty_city
            v4 += - 5 * d4_empty_city
            
        # back to city
        resource_in_cargo = 100 - unit.get_cargo_space_left()
        if resource_in_cargo > 0:
            v0 += resource_in_cargo - 10 * d0_city
            v1 += resource_in_cargo - 10 * d1_city
            v2 += resource_in_cargo - 10 * d2_city
            v3 += resource_in_cargo - 10 * d3_city
            v4 += resource_in_cargo - 10 * d4_city
        elif self.game_state.turn % 40 >= 28:
            # back if night is coming
            v0 += - 1000 * d0_city
            v1 += - 1000 * d1_city
            v2 += - 1000 * d2_city
            v3 += - 1000 * d3_city
            v4 += - 1000 * d4_city
        
        # new resource
        if resource_in_cargo < 100:
            v0 += 3 * (100 - resource_in_cargo - 10 * d0_resource)
            v1 += 3 * (100 - resource_in_cargo - 10 * d1_resource)
            v2 += 3 * (100 - resource_in_cargo - 10 * d2_resource)
            v3 += 3 * (100 - resource_in_cargo - 10 * d3_resource)
            v4 += 3 * (100 - resource_in_cargo - 10 * d4_resource)
        
        # do not go out at night
        # need to be more precise to work
        if self.game_state.turn % 40 >= 28 and resource_in_cargo == 0:
            if d0_resource > 1 and d0_city > 0:
                v0 = -10000
            if d1_resource > 1 and d1_city > 0:
                v1 = -10000
            if d2_resource > 1 and d2_city > 0:
                v2 = -10000
            if d3_resource > 1 and d3_city > 0:
                v3 = -10000
            if d4_resource > 1 and d4_city > 0:
                v4 = -10000
            
        # build is better than stay
        if len(res_list) == 0:
            res_list.append((pos_0.x, pos_0.y, v0, unit.move(Constants.DIRECTIONS.CENTER)))
        if pos_1.y >= 0:
            res_list.append((pos_1.x, pos_1.y, v1, unit.move(Constants.DIRECTIONS.NORTH)))
        if pos_2.x < self.width:
            res_list.append((pos_2.x, pos_2.y, v2, unit.move(Constants.DIRECTIONS.EAST)))
        if pos_3.y < self.height:
            res_list.append((pos_3.x, pos_3.y, v3, unit.move(Constants.DIRECTIONS.SOUTH)))
        if pos_4.x >= 0:
            res_list.append((pos_4.x, pos_4.y, v4, unit.move(Constants.DIRECTIONS.WEST)))
        # print(unit.id, res_list)

        return res_list

In [ ]:
# test for min cost max flow

action_dict = {'u_1': [(3, 27, 265, 'm u_1 c'), (3, 26, 270, 'm u_1 n'), (4, 27, 290, 'm u_1 e'), (3, 28, 295, 'm u_1 s'), (2, 27, 270, 'm u_1 w')]}
actions = []

import networkx as nx

n_units = len(action_dict.keys())
G = nx.DiGraph()
G.add_node(0, demand=-n_units)
G.add_node(1, demand=n_units)

unit_name_dict = dict()

# source -> unit
edge_list_1 = [(0, 32 * i + j + 2, 0) for i in range(32) for j in range(32)]
G.add_weighted_edges_from(edge_list_1)
# unit -> position
edge_list_2 = []
for i, k in enumerate(action_dict.keys()):
    for x, y, c, _ in action_dict[k]:
        edge_list_2.append((i + 2, 10000 + 32 * x + y, -c))
G.add_weighted_edges_from(edge_list_2)
# position -> sink
edge_list_3 = [(10000 + 32 * i + j, 1, 0) for i in range(32) for j in range(32)]
G.add_weighted_edges_from(edge_list_3)

for (i, j) in G.edges():
    G[i][j]["capacity"] = 1
    
res = nx.min_cost_flow(G)

for i, k in enumerate(action_dict.keys()):
    for x, y, c, command in action_dict[k]:
        if res[i + 2][10000 + 32 * x + y] == 1:
            if command != "":
                actions.append(command)
                
print(actions)

In [ ]:
class SmartCommand(object):
    
    def __init__(self, observation):
        self.game_state = Game()
        self.game_state._initialize(observation["updates"])
        self.game_state._update(observation["updates"][2:])
        self.game_state.id = observation.player
        self.width, self.height = self.game_state.map.width, self.game_state.map.height
        
        self.player = self.game_state.players[observation.player]
        self.opponent = self.game_state.players[(observation.player + 1) % 2]
        self.citytile_count = 0
        self.unit_count = 0
        self.resource_tiles = find_resources(self.game_state)
        
    def update(self, observation):
        self.game_state._update(observation["updates"])
        
        self.player = self.game_state.players[observation.player]
        self.opponent = self.game_state.players[(observation.player + 1) % 2]
        self.citytile_count = 0
        self.unit_count = 0
        self.resource_tiles = find_resources(self.game_state)
        
    def unit_action(self, unit, action):
        if action[0] == "move":
            return unit.move(action[1])
        elif action[0] == "city":
            self.citytile_count += 1
            return unit.build_city()
        elif action[0] == "pillage":
            return unit.pillage()
        elif action[0] == "transfer":
            return unit.transfer(action[1], action[2], action[3])  # dest_id, resourceType, amount
        elif action[0] == "traveller":
            pos = action[1]
            if unit.pos.equals(pos):
                return unit.move(Constants.DIRECTIONS.CENTER)
            else:
                unit.move(unit.pos.direction_to(pos))
        elif action[0] == "builder":
            pos_list = action[1]
            # empty tile > resource > city
            
            for pos in pos_list:
                cell = self.game_state.map.get_cell_by_pos(pos)
                if cell.citytile is None:  # already a citytile
                    break
            else:
                return self.unit_action(unit, ("citizen", ))
            if cell.has_resource():  # resource
                if unit.get_cargo_space_left() == 0:
                    new_tile = find_closest_city_tile(pos, self.player)
                    if new_tile is not None:
                        new_pos = new_tile.pos  # nearest citytile
                    else:
                        new_tile = find_closest_resources(pos, self.player, self.resource_tiles)
                        if new_tile is not None:
                            new_pos = new_tile.pos  # nearest resource
                        else:
                            new_pos = unit.pos
                    return unit.move(unit.pos.direction_to(new_pos))
                else:
                    return unit.move(unit.pos.direction_to(pos))
            else:
                if unit.get_cargo_space_left() == 0:
                    if unit.pos.equals(pos):
                        self.citytile_count += 1
                        return unit.build_city()
                    else:
                        return unit.move(unit.pos.direction_to(pos))
                else:
                    if unit.pos.equals(pos):
                        new_tile = find_closest_resources(pos, self.player, self.resource_tiles)
                        if new_tile is not None:
                            new_pos = new_tile.pos  # nearest resource
                        else:
                            nnew_pos = pos
                        return unit.move(unit.pos.direction_to(new_pos))
                    else:
                        return unit.move(unit.pos.direction_to(pos))
        elif action[0] == "citizen":
            new_tile = find_closest_city_tile(unit.pos, self.player)
            if new_tile is not None:
                new_pos = new_tile.pos  # nearest citytile
                if unit.pos.distance_to(new_pos) >= 3:  # return anyway 
                    return unit.move(unit.pos.direction_to(new_pos))
                elif unit.get_cargo_space_left() <= 10:
                    return unit.move(unit.pos.direction_to(new_pos))
                else:
                    new_tile = find_closest_resources(unit.pos, self.player, self.resource_tiles)
                    if new_tile is not None:
                        new_pos = new_tile.pos  # nearest resource
                    else:
                        new_pos = unit.pos
                    return unit.move(unit.pos.direction_to(new_pos))
            else:
                new_tile = find_closest_resources(unit.pos, self.player, self.resource_tiles)
                if new_tile is not None:
                    new_pos = new_tile.pos  # nearest resource
                else:
                    new_pos = unit.pos
                return unit.move(unit.pos.direction_to(new_pos))
        return None


    def city_tile_action(self, city_tile, action):
        if action[0] == "research":
            return city_tile.research()
        elif action[0] == "worker":
            self.unit_count += 1
            return city_tile.build_worker()
        elif action[0] == "cart":
            self.unit_count += 1
            return city_tile.build_cart()
        elif action[0] == "smart_worker":
            if self.unit_count < self.citytile_count:
                self.unit_count += 1
                return city_tile.build_worker()
            else:
                return city_tile.research()
        elif action[0] == "smart_cart":
            if self.unit_count < self.citytile_count:
                self.unit_count += 1
                return city_tile.build_cart()
            else:
                return city_tile.research()
        return None

## Semi-Manual Agent

In [ ]:
city_0_pos_list_0 = [
    Position(2, 27), Position(2, 28), Position(3, 27), Position(3, 28), 
]
city_0_pos_list_1 = [
    Position(2, 26), Position(2, 27), Position(2, 28), 
    Position(3, 26), Position(3, 27), Position(3, 28), 
    Position(4, 26), Position(4, 27), Position(4, 28), 
]
city_0_pos_list_2 = [
    Position(2, 26), Position(2, 27), Position(2, 28), Position(2, 29), 
    Position(3, 26), Position(3, 27), Position(3, 28), Position(3, 29), 
    Position(4, 26), Position(4, 27), Position(4, 28), Position(4, 29), 
    Position(5, 26), Position(5, 27), Position(5, 28), Position(5, 29), 
]

In [ ]:
sc = None
city_0 = None

def semi_manual_agent(observation, configuration):

    global sc
    global city_0
    
    ### I did edit ###
    if observation["step"] == 0:
        sc = SmartCommand(observation)
        city_0 = City(observation, city_0_pos_list_0)
    else:
        sc.update(observation)
        city_0.update(observation)
    
    if observation["step"] == 40:
        city_0.update_pos_list(city_0_pos_list_1)
    
    actions = []
    
    for unit in sc.player.units:
        sc.unit_count += 1
    for city in sc.player.cities.values():
        for city_tile in city.citytiles:
            sc.citytile_count += 1
    
    action_dict = dict()
    for unit in sc.player.units:
        # print(unit.id)
        if unit.can_act():
            if unit.id in unit_actions.keys():
                action_tuple = unit_actions[unit.id][observation["step"]]
            else:
                action_tuple = unit_actions['u_0'][observation["step"]]
            if action_tuple[0] == "citizen":
                if unit.is_worker():
                    action_dict[unit.id] = city_0.worker_action(unit)
                else:
                    action_dict[unit.id] = city_0.cart_action(unit)
            else:
                action = sc.unit_action(unit, action_tuple)
                if action is not None:
                    action_dict[unit.id] = [(unit.pos.x, unit.pos.y, 0, action)]
                else:
                    action_dict[unit.id] = [(unit.pos.x, unit.pos.y, 0, "")]               
        else:
            action_dict[unit.id] = [(unit.pos.x, unit.pos.y, 0, "")]
    # print(action_dict)
    
    # matching
    n_units = len(action_dict.keys())
    G = nx.DiGraph()
    G.add_node(0, demand=-n_units)
    G.add_node(1, demand=n_units)

    unit_name_dict = dict()

    # source -> unit
    edge_list_1 = [(0, 32 * i + j + 2, 0) for i in range(32) for j in range(32)]
    G.add_weighted_edges_from(edge_list_1)
    
    # unit -> position
    edge_list_2 = []
    for i, k in enumerate(action_dict.keys()):
        for x, y, c, _ in action_dict[k]:
            edge_list_2.append((i + 2, 10000 + 32 * x + y, -c))
    G.add_weighted_edges_from(edge_list_2)
    
    # position -> sink
    edge_list_3 = [(10000 + 32 * i + j, 1, 0) for i in range(32) for j in range(32)]
    G.add_weighted_edges_from(edge_list_3)
    
    # capacity 1
    for (i, j) in G.edges():
        G[i][j]["capacity"] = 1
        
    # citytiles have infinite capacity
    for city in sc.player.cities.values():
        for city_tile in city.citytiles:
            pos = city_tile.pos
            p_xy = 10000 + 32 * pos.x + pos.y
            G[p_xy][1]["capacity"] = 1024
    
    # resolve colision by min cost max flow 
    res = nx.min_cost_flow(G)

    for i, k in enumerate(action_dict.keys()):
        for x, y, c, command in action_dict[k]:
            if res[i + 2][10000 + 32 * x + y] == 1:
                if command != "":
                    actions.append(command)
    for city in sc.player.cities.values():
        for city_tile in city.citytiles:
            # print(city_tile.pos)
            city_tile_key = f"ct_{city_tile.pos.x}_{city_tile.pos.y}"
            if city_tile.can_act():
                action_tuple = city_tile_actions[city_tile_key][observation["step"]]
                action = sc.city_tile_action(city_tile, action_tuple)
                # print(action)
                if action is not None:
                    actions.append(action) 
    # print(actions)
    return actions

actions for citytiles, default is research

In [ ]:
city_tile_actions = {}
for i in range(32):
    for j in range(32):
        city_tile_actions[f"ct_{i}_{j}"] = [smart_worker] * 400

actions for units(workers and carts), default is random move

In [ ]:
u_0_actions = [("citizen", )] * 400

unit_actions = {
    'u_0': u_0_actions,
}

## Run and Watch
and edit commans and run and edit commands and run and ...

In [ ]:
env.reset()
_ = env.run([semi_manual_agent, "simple_agent"])
env.render(mode="ipython", width=800, height=600)

# Issues found

- do not build house by coals...